In [1]:
from ppo_eval import train_ppo
import optuna
import gym
from stable_baselines3.common.evaluation import evaluate_policy
from env.custom_randomized_hopper import register_uniform

/home/tanguy/Documents/rl_project/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_env = gym.make('CustomHopper-target-v0')
def objective_uniform(trial: optuna.Trial):
    upper = trial.suggest_int("upper", 2, 6, step=1)
    lower = trial.suggest_int("lower", 0, 3, step=1)
    if upper <= lower:
        return 0
    NAME = f"PPO-{upper}-{lower}"
    register_uniform(lower, upper, NAME+'-v0')
    train_env = gym.make(NAME+'-v0')
    model = train_ppo(train_env, 'outputs/hpppo/'+NAME+'.mdl', total_timesteps=300_000)
    mean_reward, _ = evaluate_policy(model, test_env, 500)
    train_env.close()
    return mean_reward

In [3]:
study = optuna.create_study(direction='maximize', study_name='ppo+rd hp tuning')
study.optimize(objective_uniform, n_trials=50, n_jobs=3)

[I 2024-05-25 11:39:38,820] A new study created in memory with name: ppo+rd hp tuning
[I 2024-05-25 11:39:38,987] Trial 2 finished with value: 0.0 and parameters: {'upper': 3, 'lower': 3}. Best is trial 2 with value: 0.0.
[I 2024-05-25 11:39:39,038] Trial 3 finished with value: 0.0 and parameters: {'upper': 3, 'lower': 3}. Best is trial 2 with value: 0.0.
